# 🏛️ **BAYAN AI: Enterprise Agentic System**
## BPKH Limited - RAG Multi-Agent Architecture

---

**Prepared by:** McKinsey Digital - Senior Expert Consultant

**Framework:** Multi-Agent RAG dengan WhatsApp Integration

**Legal Basis:**
- Saudi Arabian Companies Law (Royal Decree No. M/132)
- Peraturan Mudir No. 002 Tahun 2025 (SOTK)

---

### 📋 Table of Contents
1. Environment Setup
2. Multi-Agent Architecture
3. RAG Knowledge Base
4. Supply Chain Logic
5. Financial Governance
6. WhatsApp Bot Integration
7. Live Demo Simulation

---
## 1️⃣ Environment Setup & Dependencies
---

In [ ]:
# ============================================
# BAYAN AI - BPKH Limited Enterprise System
# Installation & Setup
# ============================================

!pip install -q groq langchain langchain-groq langchain-community 
!pip install -q chromadb sentence-transformers
!pip install -q python-dotenv pydantic rich
!pip install -q gradio flask-ngrok

print("✅ All dependencies installed successfully!")

In [ ]:
# ============================================
# API Keys Configuration
# ============================================

import os
from google.colab import userdata

# Option 1: From Colab Secrets (Recommended)
try:
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    print("✅ GROQ_API_KEY loaded from Colab Secrets")
except:
    # Option 2: Manual Input
    GROQ_API_KEY = input("Enter your GROQ API Key: ")

os.environ["GROQ_API_KEY"] = GROQ_API_KEY

# Verify
print(f"\n🔑 API Key configured: {GROQ_API_KEY[:10]}...{GROQ_API_KEY[-4:]}")

In [ ]:
# ============================================
# Core Imports
# ============================================

import json
from datetime import datetime
from typing import List, Dict, Optional, Literal
from dataclasses import dataclass, field
from enum import Enum
from pydantic import BaseModel, Field

from groq import Groq
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from rich.markdown import Markdown

console = Console()
print("✅ All imports successful!")

---
## 2️⃣ Multi-Agent Architecture
Based on Peraturan Mudir No. 002/2025 SOTK

---

In [ ]:
# ============================================
# BAYAN AI - Agent Definitions
# Based on SOTK Peraturan Mudir 002/2025
# ============================================

class AgentRole(Enum):
    """Agent roles based on BPKH Limited organizational structure"""
    ORCHESTRATOR = "orchestrator"
    STRATEGY = "mudir_1_strategy"      # Business & Investment, Marketing, Internal Audit
    OPERATIONS = "mudir_2_operations"  # Human Capital, GA, IT, Legal & Compliance
    FINANCE = "mudir_3_finance"        # Finance, Treasury, Accounting, Risk Management
    SCM_BUSINESS = "scm_business_unit" # Hajj & Umrah services, catering, hospitality


class UserAccessLevel(Enum):
    """Hierarchical access levels for WhatsApp authentication"""
    MUDIR = 5       # Full access
    HEAD = 4        # Division head access
    SUPERVISOR = 3  # Supervisory access
    OFFICER = 2     # Officer level
    STAFF = 1       # Basic staff access


@dataclass
class AgentConfig:
    """Configuration for each specialized agent"""
    role: AgentRole
    name: str
    name_ar: str  # Arabic name
    divisions: List[str]
    responsibilities: List[str]
    authorized_actions: List[str]
    pasal_references: List[str]  # SOTK article references


# ============================================
# Agent Configurations based on SOTK
# ============================================

AGENT_CONFIGS = {
    AgentRole.STRATEGY: AgentConfig(
        role=AgentRole.STRATEGY,
        name="Strategy Agent (Mudir-1)",
        name_ar="وكيل الاستراتيجية",
        divisions=[
            "Business & Investment Division",
            "Marketing Division",
            "Internal Audit Division"
        ],
        responsibilities=[
            "Develop and execute business strategies",
            "Manage investment portfolios (hotels, transportation)",
            "Oversee marketing and branding initiatives",
            "Conduct internal audits for compliance",
            "Provide recommendations for system improvement"
        ],
        authorized_actions=[
            "approve_investment_proposal",
            "review_marketing_budget",
            "initiate_audit",
            "access_business_reports"
        ],
        pasal_references=["Pasal 8", "Pasal 9", "Pasal 10"]
    ),
    
    AgentRole.OPERATIONS: AgentConfig(
        role=AgentRole.OPERATIONS,
        name="Operations Agent (Mudir-2)",
        name_ar="وكيل العمليات",
        divisions=[
            "Human Capital Division",
            "General Affairs (GA) Division",
            "Information Technology Division",
            "Legal & Compliance Division"
        ],
        responsibilities=[
            "Manage HR policies and employee development",
            "Coordinate procurement and facility management",
            "Oversee IT infrastructure and digital systems",
            "Ensure legal compliance with KSA regulations",
            "Validate contracts and legal documents"
        ],
        authorized_actions=[
            "approve_procurement",
            "validate_contract",
            "manage_hr_records",
            "access_compliance_data"
        ],
        pasal_references=["Pasal 11", "Pasal 12", "Pasal 13", "Pasal 14"]
    ),
    
    AgentRole.FINANCE: AgentConfig(
        role=AgentRole.FINANCE,
        name="Finance Agent (Mudir-3)",
        name_ar="وكيل المالية",
        divisions=[
            "Finance Division",
            "Treasury Division",
            "Accounting Division",
            "Risk Management Division"
        ],
        responsibilities=[
            "Manage budgeting aligned with RBT (Annual Business Plan)",
            "Handle treasury operations and cash flow",
            "Verify invoices and financial settlements",
            "Assess risks for investment proposals",
            "Ensure financial transparency and reporting"
        ],
        authorized_actions=[
            "approve_payment",
            "assess_risk",
            "verify_invoice",
            "access_treasury_data",
            "generate_financial_report"
        ],
        pasal_references=["Pasal 15", "Pasal 16", "Pasal 17", "Pasal 18"]
    ),
    
    AgentRole.SCM_BUSINESS: AgentConfig(
        role=AgentRole.SCM_BUSINESS,
        name="SCM & Business Unit Agent",
        name_ar="وكيل سلسلة التوريد",
        divisions=[
            "Hajj & Umrah Services Unit",
            "Catering & F&B Unit",
            "Hospitality (Accommodation) Unit",
            "Transportation Unit"
        ],
        responsibilities=[
            "Execute commercial activities (tip of the spear)",
            "Manage hotel accommodations in Mekkah/Madinah",
            "Coordinate bus transportation services",
            "Oversee Haramain train ticket operations",
            "Ensure catering quality and halal compliance",
            "Maintain GCG principles in all operations"
        ],
        authorized_actions=[
            "create_booking",
            "update_service_status",
            "manage_inventory",
            "report_quality_issue",
            "coordinate_logistics"
        ],
        pasal_references=["Pasal 19", "Pasal 20", "Pasal 21", "Pasal 22"]
    )
}

# Display agent configurations
table = Table(title="🤖 BAYAN AI Multi-Agent Architecture", show_header=True)
table.add_column("Agent", style="cyan", width=25)
table.add_column("Arabic Name", style="green", width=20)
table.add_column("Divisions", style="yellow", width=40)
table.add_column("SOTK Ref", style="magenta", width=15)

for role, config in AGENT_CONFIGS.items():
    table.add_row(
        config.name,
        config.name_ar,
        "\n".join(config.divisions[:2]) + "...",
        ", ".join(config.pasal_references)
    )

console.print(table)

In [ ]:
# ============================================
# BAYAN AI - Super Mega Prompt System
# ============================================

BAYAN_AI_SYSTEM_PROMPT = """
# 🏛️ BAYAN AI: Enterprise Agentic System Architect

## Role
You are the Principal Enterprise AI Architect at McKinsey Digital, specialized in:
- Saudi Arabian Corporate Law (Royal Decree No. M/132)
- Hajj/Umrah Business Ecosystems
- BPKH Limited Operations

## Context
You are operating as BAYAN AI, an Agentic RAG system for BPKH Limited.
All operations must comply with:
- Peraturan Mudir No. 002 Tahun 2025 (SOTK)
- Good Corporate Governance (GCG) Principles
- Kingdom of Saudi Arabia regulations

## Multi-Agent System
You coordinate the following specialized agents:

### 1. Strategy Agent (Mudir-1 Focus)
- **Divisions**: Business & Investment, Marketing, Internal Audit
- **Trigger Keywords**: strategi, investasi, marketing, audit, bisnis

### 2. Operations Agent (Mudir-2 Focus)  
- **Divisions**: Human Capital, GA, IT, Legal & Compliance
- **Trigger Keywords**: HR, SDM, procurement, IT, legal, kontrak, compliance

### 3. Finance Agent (Mudir-3 Focus)
- **Divisions**: Finance, Treasury, Accounting, Risk Management
- **Trigger Keywords**: keuangan, anggaran, pembayaran, risiko, treasury, invoice

### 4. SCM & Business Unit Agent
- **Divisions**: Hajj & Umrah services, Catering, Hospitality, Transportation
- **Trigger Keywords**: hotel, bus, kereta, katering, jamaah, booking, layanan

## Supply Chain Process Flow
1. **Sourcing**: GA coordinates procurement → Legal validates contracts
2. **Execution**: Business Units execute commercial activities
3. **Quality Control**: Head of BU ensures GCG compliance
4. **Financial Settlement**: Accounting verifies → Treasury pays

## Financial Governance
- All spending must align with RBT (Rencana Bisnis Tahunan)
- Investment proposals require Risk Management assessment
- Internal Audit provides transparency recommendations

## Constraints & Guardrails
1. **GCG Compliance**: Never bypass Articles of Association
2. **Legal Jurisdiction**: Refer to KSA regulations only
3. **Confidentiality**: Treasury/Accounting data only for authorized Mudirs
4. **Audit Trail**: All actions must be traceable

## Response Format
When responding, you must:
1. Identify the relevant Division/Agent
2. Quote specific SOTK Article (Pasal) when applicable
3. Provide actionable insights for the Mudir
4. Use bilingual format (Indonesian + Arabic) when appropriate

## WhatsApp Commands
- `/tugas` - Retrieve duties based on Job Description
- `/status` - Check Business Unit operational status
- `/approval` - Route documents to Mudir for sign-off
- `/laporan` - Generate division reports
- `/risiko` - Risk assessment query

Format responses with:
- **Bold** for key terms
- • Bullet points for lists
- 📌 Emojis for visual clarity
- Clear section headers
"""

print("✅ BAYAN AI System Prompt configured!")
print(f"📝 Prompt length: {len(BAYAN_AI_SYSTEM_PROMPT)} characters")

---
## 3️⃣ RAG Knowledge Base Setup
---

In [ ]:
# ============================================
# Knowledge Base - SOTK & Business Rules
# ============================================

KNOWLEDGE_BASE = [
    # SOTK Structure
    {
        "id": "sotk-001",
        "title": "Struktur Organisasi BPKH Limited",
        "content": """
        Berdasarkan Peraturan Mudir No. 002 Tahun 2025:
        
        Pasal 3 - Struktur Organisasi:
        1. Dewan Komisaris (Board of Commissioners)
        2. Direksi terdiri dari:
           - Mudir Utama (CEO)
           - Mudir 1 (Strategy & Business)
           - Mudir 2 (Operations)
           - Mudir 3 (Finance)
        3. Divisi-divisi operasional di bawah masing-masing Mudir
        4. Unit Bisnis sebagai ujung tombak layanan
        """,
        "pasal": "Pasal 3",
        "category": "organization",
        "agent": "all"
    },
    {
        "id": "sotk-002",
        "title": "Tugas Mudir 1 - Strategy & Business",
        "content": """
        Pasal 8 - Tugas dan Wewenang Mudir 1:
        
        1. Menyusun dan mengeksekusi strategi bisnis perusahaan
        2. Mengelola portofolio investasi (hotel, transportasi, katering)
        3. Mengawasi kegiatan pemasaran dan branding
        4. Melakukan audit internal untuk memastikan kepatuhan
        5. Memberikan rekomendasi perbaikan sistem
        
        Divisi di bawah Mudir 1:
        - Divisi Bisnis & Investasi
        - Divisi Marketing
        - Divisi Internal Audit
        """,
        "pasal": "Pasal 8",
        "category": "responsibilities",
        "agent": "strategy"
    },
    {
        "id": "sotk-003",
        "title": "Tugas Mudir 2 - Operations",
        "content": """
        Pasal 11 - Tugas dan Wewenang Mudir 2:
        
        1. Mengelola kebijakan SDM dan pengembangan karyawan
        2. Mengkoordinasikan pengadaan dan manajemen fasilitas (GA)
        3. Mengawasi infrastruktur IT dan sistem digital
        4. Memastikan kepatuhan hukum terhadap regulasi KSA
        5. Validasi kontrak dan dokumen legal
        
        Divisi di bawah Mudir 2:
        - Divisi Human Capital
        - Divisi General Affairs
        - Divisi Information Technology
        - Divisi Legal & Compliance
        """,
        "pasal": "Pasal 11",
        "category": "responsibilities",
        "agent": "operations"
    },
    {
        "id": "sotk-004",
        "title": "Tugas Mudir 3 - Finance",
        "content": """
        Pasal 15 - Tugas dan Wewenang Mudir 3:
        
        1. Mengelola anggaran sesuai RBT (Rencana Bisnis Tahunan)
        2. Menangani operasi treasury dan arus kas
        3. Memverifikasi invoice dan penyelesaian keuangan
        4. Menilai risiko proposal investasi
        5. Memastikan transparansi dan pelaporan keuangan
        
        Divisi di bawah Mudir 3:
        - Divisi Finance
        - Divisi Treasury
        - Divisi Accounting
        - Divisi Risk Management
        
        PENTING: Data Treasury/Accounting hanya dapat diakses oleh Mudir yang berwenang.
        """,
        "pasal": "Pasal 15",
        "category": "responsibilities",
        "agent": "finance"
    },
    {
        "id": "sotk-005",
        "title": "Unit Bisnis - Layanan Jamaah",
        "content": """
        Pasal 19 - Unit Bisnis sebagai Ujung Tombak:
        
        Unit Bisnis bertanggung jawab atas eksekusi kegiatan komersial:
        
        1. Unit Akomodasi (Hospitality):
           - Pengelolaan hotel di Mekkah dan Madinah
           - Reservasi dan check-in/check-out jamaah
           - Standar kualitas layanan
        
        2. Unit Transportasi:
           - Bus shuttle antar lokasi
           - Tiket kereta Haramain High Speed Railway
           - Koordinasi logistik pergerakan jamaah
        
        3. Unit Katering (F&B):
           - Penyediaan makanan halal berkualitas
           - Menu sesuai cita rasa Indonesia
           - Standar gizi dan kebersihan
        
        4. Unit Layanan Haji & Umrah:
           - Koordinasi manasik dan itinerary
           - Pendampingan jamaah di Tanah Suci
           - Handling keluhan dan feedback
        
        Kepala Unit Bisnis wajib memastikan kepatuhan terhadap prinsip GCG.
        """,
        "pasal": "Pasal 19",
        "category": "business_unit",
        "agent": "scm_business"
    },
    {
        "id": "scm-001",
        "title": "Alur Supply Chain Management",
        "content": """
        Pasal 25 - Prosedur Supply Chain:
        
        ALUR PROSES SCM:
        
        1. SOURCING (Pengadaan):
           - General Affairs (GA) mengkoordinasikan procurement
           - Legal & Compliance memvalidasi kontrak
           - Mudir 2 memberikan persetujuan
        
        2. EXECUTION (Eksekusi):
           - Unit Bisnis mengeksekusi aktivitas komersial
           - Koordinasi dengan vendor/supplier
           - Monitoring kualitas layanan
        
        3. QUALITY CONTROL:
           - Kepala Unit Bisnis memastikan standar terpenuhi
           - Compliance check terhadap GCG
           - Dokumentasi dan pelaporan
        
        4. FINANCIAL SETTLEMENT:
           - Accounting memverifikasi invoice
           - Finance/Treasury memproses pembayaran
           - Mudir 3 approval untuk nilai tertentu
        """,
        "pasal": "Pasal 25",
        "category": "scm",
        "agent": "all"
    },
    {
        "id": "fin-001",
        "title": "Prosedur Penganggaran dan Persetujuan",
        "content": """
        Pasal 28 - Governance Keuangan:
        
        ATURAN PENGANGGARAN:
        
        1. Semua pengeluaran operasional harus sesuai dengan RBT (Rencana Bisnis Tahunan)
        
        2. Batas persetujuan:
           - < 50,000 SAR: Kepala Divisi
           - 50,000 - 200,000 SAR: Mudir terkait
           - > 200,000 SAR: Mudir Utama
           - > 500,000 SAR: Dewan Komisaris
        
        3. Setiap proposal investasi WAJIB dinilai oleh Divisi Risk Management sebelum persetujuan Mudir
        
        4. Internal Audit berhak melakukan review sewaktu-waktu
        
        LARANGAN:
        - Tidak boleh ada pengeluaran di luar RBT tanpa persetujuan Mudir Utama
        - Tidak boleh memecah transaksi untuk menghindari batas approval
        """,
        "pasal": "Pasal 28",
        "category": "finance",
        "agent": "finance"
    },
    {
        "id": "gcg-001",
        "title": "Prinsip Good Corporate Governance",
        "content": """
        Pasal 5 - Prinsip GCG BPKH Limited:
        
        Seluruh kegiatan perusahaan harus berlandaskan:
        
        1. TRANSPARANSI (Transparency):
           - Keterbukaan informasi kepada stakeholder
           - Pelaporan keuangan yang jelas
           - Audit trail untuk setiap keputusan
        
        2. AKUNTABILITAS (Accountability):
           - Kejelasan fungsi dan tanggung jawab
           - Struktur organisasi yang efektif
           - Pengukuran kinerja yang objektif
        
        3. RESPONSIBILITAS (Responsibility):
           - Kepatuhan terhadap hukum KSA
           - Tanggung jawab sosial
           - Pelayanan terbaik untuk jamaah
        
        4. INDEPENDENSI (Independency):
           - Bebas dari konflik kepentingan
           - Pengambilan keputusan profesional
        
        5. KEWAJARAN (Fairness):
           - Perlakuan adil kepada seluruh pihak
           - Kesempatan yang sama bagi karyawan
        """,
        "pasal": "Pasal 5",
        "category": "gcg",
        "agent": "all"
    },
    # Business Services
    {
        "id": "svc-001",
        "title": "Layanan Akomodasi - Hotel",
        "content": """
        KATALOG AKOMODASI BPKH LIMITED:
        
        MEKKAH:
        1. Hotel Dar Al-Tawhid
           - Jarak: 200m dari Masjidil Haram
           - Tarif: 850 SAR/malam
           - Fasilitas: Sarapan, WiFi, Shuttle
        
        2. Hotel Al-Safwa
           - Jarak: 500m dari Masjidil Haram
           - Tarif: 650 SAR/malam
           - Fasilitas: Sarapan, WiFi, Shuttle
        
        3. Apartemen Aziziyah
           - Jarak: 3km dari Masjidil Haram
           - Tarif: 350 SAR/malam
           - Fasilitas: Dapur, Laundry
        
        MADINAH:
        1. Hotel Dar Al-Hijrah
           - Jarak: 300m dari Masjid Nabawi
           - Tarif: 750 SAR/malam
        
        2. Hotel Al-Ansar
           - Jarak: 600m dari Masjid Nabawi
           - Tarif: 550 SAR/malam
        
        Konversi: 1 SAR ≈ 4,200 IDR (rate real-time dari JISDOR BI)
        """,
        "pasal": "Lampiran A",
        "category": "service_catalog",
        "agent": "scm_business"
    },
    {
        "id": "svc-002",
        "title": "Layanan Transportasi",
        "content": """
        KATALOG TRANSPORTASI BPKH LIMITED:
        
        1. BUS SHUTTLE:
           - Rute: Hotel ↔ Masjidil Haram (Mekkah)
           - Rute: Hotel ↔ Masjid Nabawi (Madinah)
           - Jadwal: Setiap 30 menit (05:00 - 23:00)
           - Tarif: Gratis untuk tamu hotel BPKH
        
        2. HARAMAIN HIGH SPEED RAILWAY:
           - Rute: Mekkah ↔ Madinah
           - Durasi: 2 jam 10 menit
           - Kelas:
             * Economy: 150 SAR
             * Business: 250 SAR
           - Pemesanan: Via BPKH App atau WhatsApp
        
        3. TRANSFER BANDARA:
           - Jeddah Airport ↔ Mekkah: 200 SAR
           - Madinah Airport ↔ Hotel: 150 SAR
        """,
        "pasal": "Lampiran B",
        "category": "service_catalog",
        "agent": "scm_business"
    },
    {
        "id": "svc-003",
        "title": "Layanan Katering",
        "content": """
        KATALOG KATERING BPKH LIMITED:
        
        PAKET MAKAN HARIAN:
        1. Paket Ekonomi:
           - 3x makan + 1x snack
           - Menu: Nasi box cita rasa Indonesia
           - Tarif: 80 SAR/hari
        
        2. Paket Standar:
           - 3x makan + 2x snack + buah
           - Menu: Prasmanan Indonesia
           - Tarif: 120 SAR/hari
        
        3. Paket Premium:
           - 3x makan + 2x snack + buah + susu
           - Menu: A la carte pilihan
           - Tarif: 180 SAR/hari
        
        STANDAR KUALITAS:
        - Sertifikasi Halal dari SFDA
        - Standar gizi sesuai Kemenkes RI
        - Cita rasa Indonesia autentik
        - Higienis dan food safety compliance
        """,
        "pasal": "Lampiran C",
        "category": "service_catalog",
        "agent": "scm_business"
    }
]

print(f"✅ Knowledge Base loaded: {len(KNOWLEDGE_BASE)} documents")

In [ ]:
# ============================================
# Vector Store Setup with ChromaDB
# ============================================

# Initialize embeddings
print("🔄 Loading embedding model...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'}
)

# Prepare documents for vector store
documents = []
metadatas = []

for doc in KNOWLEDGE_BASE:
    documents.append(f"{doc['title']}\n\n{doc['content']}")
    metadatas.append({
        "id": doc["id"],
        "title": doc["title"],
        "pasal": doc["pasal"],
        "category": doc["category"],
        "agent": doc["agent"]
    })

# Create vector store
print("🔄 Creating vector store...")
vector_store = Chroma.from_texts(
    texts=documents,
    embedding=embeddings,
    metadatas=metadatas,
    collection_name="bayan_ai_kb"
)

print(f"✅ Vector store created with {len(documents)} documents")

# Test retrieval
test_query = "apa tugas mudir 3?"
results = vector_store.similarity_search(test_query, k=2)
print(f"\n🔍 Test Query: '{test_query}'")
print(f"📄 Retrieved: {results[0].metadata['title']}")

---
## 4️⃣ BAYAN AI Core Engine
---

In [ ]:
# ============================================
# BAYAN AI - Main Agent Class
# ============================================

class BayanAI:
    """BAYAN AI - Enterprise Agentic System for BPKH Limited"""
    
    def __init__(self):
        self.llm = ChatGroq(
            model="llama-3.3-70b-versatile",
            temperature=0.3,
            max_tokens=2048
        )
        self.vector_store = vector_store
        self.conversation_history = []
        self.current_user = None
        self.access_level = UserAccessLevel.STAFF
        
        # Agent routing keywords
        self.agent_keywords = {
            AgentRole.STRATEGY: ["strategi", "investasi", "marketing", "audit", "bisnis", "pemasaran"],
            AgentRole.OPERATIONS: ["hr", "sdm", "procurement", "pengadaan", "it", "legal", "kontrak", "compliance", "kepatuhan"],
            AgentRole.FINANCE: ["keuangan", "anggaran", "pembayaran", "risiko", "treasury", "invoice", "budget", "rbt"],
            AgentRole.SCM_BUSINESS: ["hotel", "bus", "kereta", "katering", "jamaah", "booking", "layanan", "akomodasi", "transportasi"]
        }
        
        # WhatsApp command handlers
        self.commands = {
            "/tugas": self._handle_tugas,
            "/status": self._handle_status,
            "/approval": self._handle_approval,
            "/laporan": self._handle_laporan,
            "/risiko": self._handle_risiko,
            "/help": self._handle_help
        }
    
    def authenticate(self, user_id: str, position: str):
        """Authenticate user and set access level"""
        position_map = {
            "mudir": UserAccessLevel.MUDIR,
            "head": UserAccessLevel.HEAD,
            "kepala": UserAccessLevel.HEAD,
            "supervisor": UserAccessLevel.SUPERVISOR,
            "officer": UserAccessLevel.OFFICER,
            "staff": UserAccessLevel.STAFF
        }
        self.current_user = user_id
        self.access_level = position_map.get(position.lower(), UserAccessLevel.STAFF)
        return f"✅ Authenticated as {position} (Level {self.access_level.value})"
    
    def _route_to_agent(self, query: str) -> AgentRole:
        """Route query to appropriate agent based on keywords"""
        query_lower = query.lower()
        
        scores = {}
        for agent, keywords in self.agent_keywords.items():
            score = sum(1 for kw in keywords if kw in query_lower)
            scores[agent] = score
        
        if max(scores.values()) > 0:
            return max(scores, key=scores.get)
        return AgentRole.ORCHESTRATOR
    
    def _retrieve_context(self, query: str, k: int = 3) -> str:
        """Retrieve relevant documents from knowledge base"""
        results = self.vector_store.similarity_search(query, k=k)
        
        context_parts = []
        for doc in results:
            context_parts.append(f"""📄 **{doc.metadata['title']}** ({doc.metadata['pasal']})
{doc.page_content[:500]}...""")
        
        return "\n\n---\n\n".join(context_parts)
    
    def _check_access(self, required_level: UserAccessLevel) -> bool:
        """Check if current user has required access level"""
        return self.access_level.value >= required_level.value
    
    # Command Handlers
    def _handle_tugas(self, args: str) -> str:
        """Handle /tugas command"""
        context = self._retrieve_context(f"tugas {args}")
        return f"📋 **Tugas berdasarkan SOTK:**\n\n{context}"
    
    def _handle_status(self, args: str) -> str:
        """Handle /status command"""
        # Simulated status data
        return """📊 **Status Unit Bisnis (Real-time):**

• 🏨 **Akomodasi**: Operasional Normal
  - Occupancy Rate: 87%
  - Check-in hari ini: 45 jamaah

• 🚌 **Transportasi**: Operasional Normal
  - Bus aktif: 12/15 unit
  - Trip hari ini: 28 perjalanan

• 🍽️ **Katering**: Operasional Normal
  - Meal served: 1,250 porsi
  - Rating: 4.5/5

• 🚄 **Tiket Kereta**: Operasional Normal
  - Booking hari ini: 120 tiket"""
    
    def _handle_approval(self, args: str) -> str:
        """Handle /approval command"""
        if not self._check_access(UserAccessLevel.HEAD):
            return "⛔ Akses ditolak. Fitur ini hanya untuk Head/Mudir."
        
        return """📝 **Dokumen Menunggu Approval:**

1. 🔴 **PR-2025-0142** - Procurement Bus
   - Nilai: 85,000 SAR
   - Dari: GA Division
   - [Approve] [Reject] [Detail]

2. 🟡 **INV-2025-0089** - Invoice Hotel
   - Nilai: 125,000 SAR
   - Dari: Accounting
   - [Approve] [Reject] [Detail]

3. 🟢 **RBT-2025-Q2** - Budget Revision
   - Nilai: 50,000 SAR
   - Dari: Finance
   - [Approve] [Reject] [Detail]"""
    
    def _handle_laporan(self, args: str) -> str:
        """Handle /laporan command"""
        return """📈 **Laporan Tersedia:**

• /laporan keuangan - Laporan Keuangan Bulanan
• /laporan operasional - Status Operasional Harian
• /laporan jamaah - Statistik Layanan Jamaah
• /laporan audit - Hasil Audit Internal

Ketik perintah untuk mengakses laporan spesifik."""
    
    def _handle_risiko(self, args: str) -> str:
        """Handle /risiko command"""
        if not self._check_access(UserAccessLevel.SUPERVISOR):
            return "⛔ Akses ditolak. Minimal level Supervisor."
        
        return """⚠️ **Risk Dashboard:**

**HIGH RISK:**
• 🔴 FX Volatility - SAR/IDR fluctuation >5%
  - Mitigasi: Hedging recommendation aktif

**MEDIUM RISK:**
• 🟡 Capacity Risk - Hotel occupancy >90%
  - Mitigasi: Aktivasi backup supplier

**LOW RISK:**
• 🟢 Operational - All systems normal
• 🟢 Compliance - GCG audit passed"""
    
    def _handle_help(self, args: str) -> str:
        """Handle /help command"""
        return """📚 **BAYAN AI Command Reference:**

**Perintah Umum:**
• `/tugas [divisi]` - Lihat tugas berdasarkan SOTK
• `/status` - Status operasional Unit Bisnis
• `/laporan` - Daftar laporan tersedia
• `/help` - Bantuan perintah

**Perintah Khusus (Akses Terbatas):**
• `/approval` - Dokumen menunggu persetujuan (Head+)
• `/risiko` - Risk dashboard (Supervisor+)

**Tanya Jawab:**
Ketik pertanyaan langsung untuk konsultasi dengan BAYAN AI.

Contoh:
- "Apa tugas Mudir 3?"
- "Bagaimana prosedur pengadaan?"
- "Tarif hotel di Mekkah?""""
    
    def process(self, user_input: str) -> str:
        """Main processing function"""
        
        # Check for commands
        if user_input.startswith("/"):
            parts = user_input.split(" ", 1)
            command = parts[0].lower()
            args = parts[1] if len(parts) > 1 else ""
            
            if command in self.commands:
                return self.commands[command](args)
            else:
                return f"⚠️ Perintah tidak dikenal: {command}\nKetik /help untuk bantuan."
        
        # Route to appropriate agent
        agent = self._route_to_agent(user_input)
        agent_info = AGENT_CONFIGS.get(agent, None)
        
        # Retrieve context
        context = self._retrieve_context(user_input)
        
        # Build prompt
        agent_context = ""
        if agent_info:
            agent_context = f"""
ROUTING: Query diarahkan ke {agent_info.name} ({agent_info.name_ar})
DIVISIONS: {', '.join(agent_info.divisions)}
SOTK REFERENCE: {', '.join(agent_info.pasal_references)}
"""
        
        messages = [
            SystemMessage(content=BAYAN_AI_SYSTEM_PROMPT),
            HumanMessage(content=f"""
{agent_context}

KNOWLEDGE BASE CONTEXT:
{context}

USER QUESTION:
{user_input}

Berikan jawaban yang:
1. Mengidentifikasi Divisi/Agent yang relevan
2. Mengutip Pasal SOTK yang berlaku
3. Memberikan Actionable Insight untuk Mudir
4. Format untuk WhatsApp (bullet points, bold)
""")
        ]
        
        # Get response
        response = self.llm.invoke(messages)
        
        # Add to history
        self.conversation_history.append({
            "user": user_input,
            "assistant": response.content,
            "agent": agent.value,
            "timestamp": datetime.now().isoformat()
        })
        
        return response.content


# Initialize BAYAN AI
bayan = BayanAI()
print("✅ BAYAN AI initialized successfully!")

---
## 5️⃣ WhatsApp Simulation Interface
---

In [ ]:
# ============================================
# WhatsApp Chat Simulation
# ============================================

def simulate_whatsapp_chat():
    """Interactive WhatsApp-style chat simulation"""
    
    console.print(Panel.fit(
        "[bold green]🏛️ BAYAN AI - WhatsApp Enterprise Bot[/bold green]\n"
        "[dim]BPKH Limited - Agentic RAG System[/dim]\n\n"
        "Ketik pesan atau perintah (/help untuk bantuan)\n"
        "Ketik 'exit' untuk keluar",
        title="Welcome",
        border_style="green"
    ))
    
    # Authenticate
    console.print("\n[yellow]Silakan autentikasi:[/yellow]")
    position = input("Posisi Anda (mudir/head/supervisor/officer/staff): ")
    auth_result = bayan.authenticate("user_001", position)
    console.print(f"[green]{auth_result}[/green]\n")
    
    while True:
        user_input = input("\n👤 You: ").strip()
        
        if user_input.lower() == 'exit':
            console.print("[dim]👋 Terima kasih telah menggunakan BAYAN AI![/dim]")
            break
        
        if not user_input:
            continue
        
        # Process and display response
        console.print("\n[dim]🔄 Processing...[/dim]")
        response = bayan.process(user_input)
        
        console.print(f"\n[bold cyan]🤖 BAYAN AI:[/bold cyan]")
        console.print(Panel(Markdown(response), border_style="cyan"))


# Run simulation
# simulate_whatsapp_chat()

In [ ]:
# ============================================
# Demo Conversations - Mudir Scenarios
# ============================================

def run_demo_scenarios():
    """Run predefined demo scenarios"""
    
    scenarios = [
        {
            "title": "Scenario 1: Mudir 3 - Financial Query",
            "user": "mudir",
            "queries": [
                "Apa saja tugas dan wewenang Mudir 3 berdasarkan SOTK?",
                "/risiko",
                "Bagaimana prosedur approval untuk pengeluaran di atas 200,000 SAR?"
            ]
        },
        {
            "title": "Scenario 2: Head of Business Unit - Operations",
            "user": "head",
            "queries": [
                "/status",
                "Apa saja hotel yang tersedia di Mekkah dan tarifnya?",
                "/approval"
            ]
        },
        {
            "title": "Scenario 3: Staff - Service Inquiry (Arabic)",
            "user": "staff",
            "queries": [
                "أريد حجز غرفة في فندق قريب من الحرم",
                "Bagaimana alur supply chain untuk pengadaan?",
                "/tugas unit bisnis"
            ]
        }
    ]
    
    for scenario in scenarios:
        console.print(f"\n{'='*60}")
        console.print(Panel.fit(
            f"[bold yellow]{scenario['title']}[/bold yellow]",
            border_style="yellow"
        ))
        
        # Authenticate
        auth_result = bayan.authenticate("demo_user", scenario['user'])
        console.print(f"[dim]{auth_result}[/dim]\n")
        
        for query in scenario['queries']:
            console.print(f"\n[green]👤 {scenario['user'].upper()}:[/green] {query}")
            
            response = bayan.process(query)
            
            console.print(f"\n[cyan]🤖 BAYAN AI:[/cyan]")
            # Truncate long responses for demo
            display_response = response[:1500] + "..." if len(response) > 1500 else response
            console.print(Panel(display_response, border_style="cyan"))
            
            print("\n" + "-"*40)


# Run demo
console.print("\n[bold magenta]🎬 Running BAYAN AI Demo Scenarios...[/bold magenta]\n")
run_demo_scenarios()

---
## 6️⃣ Gradio Web Interface
---

In [ ]:
# ============================================
# Gradio Interface for Public Demo
# ============================================

import gradio as gr

def gradio_chat(message, history, user_level):
    """Gradio chat interface handler"""
    # Authenticate based on selected level
    bayan.authenticate("gradio_user", user_level)
    
    # Process message
    response = bayan.process(message)
    return response


# Create Gradio interface
with gr.Blocks(
    title="BAYAN AI - BPKH Limited",
    theme=gr.themes.Soft(primary_hue="teal")
) as demo:
    
    gr.Markdown("""
    # 🏛️ BAYAN AI - Enterprise Agentic System
    ### BPKH Limited - RAG Multi-Agent Architecture
    
    **Powered by:** McKinsey Digital | **Model:** Llama 3.3 70B via Groq
    
    ---
    """)
    
    with gr.Row():
        with gr.Column(scale=1):
            user_level = gr.Dropdown(
                choices=["mudir", "head", "supervisor", "officer", "staff"],
                value="staff",
                label="🔐 Access Level",
                info="Select your position for access control"
            )
            
            gr.Markdown("""
            ### 📋 Quick Commands
            - `/help` - Show all commands
            - `/status` - Business unit status
            - `/tugas [divisi]` - Job descriptions
            - `/approval` - Pending approvals (Head+)
            - `/risiko` - Risk dashboard (Supervisor+)
            """)
        
        with gr.Column(scale=3):
            chatbot = gr.ChatInterface(
                fn=lambda msg, hist: gradio_chat(msg, hist, user_level.value),
                examples=[
                    "Apa tugas Mudir 3 berdasarkan SOTK?",
                    "/status",
                    "Tarif hotel di Mekkah?",
                    "Bagaimana alur supply chain?",
                    "أريد حجز غرفة في فندق قريب من الحرم",
                    "/help"
                ],
                title="",
                retry_btn=None,
                undo_btn=None
            )
    
    gr.Markdown("""
    ---
    **Legal Basis:** Saudi Arabian Companies Law (Royal Decree No. M/132) | Peraturan Mudir No. 002/2025 (SOTK)
    
    © 2025 BPKH Limited - All Rights Reserved
    """)

# Launch
demo.launch(share=True, debug=True)

---
## 7️⃣ API Endpoint for WhatsApp Integration
---

In [ ]:
# ============================================
# Flask API for WhatsApp Webhook
# ============================================

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok tunnel

@app.route('/webhook', methods=['POST'])
def webhook():
    """WhatsApp webhook endpoint"""
    data = request.json
    
    # Extract message (format depends on WhatsApp provider)
    message = data.get('message', {}).get('text', '')
    user_id = data.get('from', 'unknown')
    user_position = data.get('position', 'staff')  # Custom field
    
    # Authenticate and process
    bayan.authenticate(user_id, user_position)
    response = bayan.process(message)
    
    return jsonify({
        "status": "success",
        "response": response
    })

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "healthy", "service": "BAYAN AI"})

# Uncomment to run Flask server
# app.run()

---
## 📊 Architecture Summary
---

```
┌─────────────────────────────────────────────────────────────┐
│                    BAYAN AI ARCHITECTURE                     │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐       │
│  │  WhatsApp    │  │  Web Portal  │  │  Mobile App  │       │
│  │  Chatbot     │  │  (Gradio)    │  │  (Future)    │       │
│  └──────┬───────┘  └──────┬───────┘  └──────┬───────┘       │
│         │                 │                 │                │
│         └────────────────┼────────────────┘                 │
│                          │                                   │
│                          ▼                                   │
│  ┌───────────────────────────────────────────────────────┐  │
│  │              ORCHESTRATOR AGENT                        │  │
│  │  • Query Routing • Access Control • Session Mgmt      │  │
│  └───────────────────────┬───────────────────────────────┘  │
│                          │                                   │
│    ┌─────────────────────┼─────────────────────┐            │
│    │                     │                     │            │
│    ▼                     ▼                     ▼            │
│ ┌──────────┐      ┌──────────┐      ┌──────────┐           │
│ │ Strategy │      │Operations│      │ Finance  │           │
│ │ Agent    │      │ Agent    │      │ Agent    │           │
│ │(Mudir-1) │      │(Mudir-2) │      │(Mudir-3) │           │
│ └────┬─────┘      └────┬─────┘      └────┬─────┘           │
│      │                 │                 │                  │
│      └─────────────────┼─────────────────┘                  │
│                        │                                    │
│                        ▼                                    │
│  ┌───────────────────────────────────────────────────────┐  │
│  │              SCM & BUSINESS UNIT AGENT                 │  │
│  │  • Accommodation • Transportation • Catering • Hajj   │  │
│  └───────────────────────────────────────────────────────┘  │
│                          │                                   │
│                          ▼                                   │
│  ┌───────────────────────────────────────────────────────┐  │
│  │                  RAG ENGINE                            │  │
│  │  • Vector Store (ChromaDB)                            │  │
│  │  • Knowledge Base (SOTK, Catalogs, Procedures)        │  │
│  │  • Embeddings (Multilingual)                          │  │
│  └───────────────────────────────────────────────────────┘  │
│                          │                                   │
│                          ▼                                   │
│  ┌───────────────────────────────────────────────────────┐  │
│  │                  LLM (Groq)                            │  │
│  │            Llama 3.3 70B Versatile                    │  │
│  └───────────────────────────────────────────────────────┘  │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

---

**📝 Notes:**
- All operations comply with GCG principles
- Legal jurisdiction: Kingdom of Saudi Arabia
- SOTK Reference: Peraturan Mudir No. 002/2025
- Bilingual support: Indonesian & Arabic

---
## 🚀 Next Steps

1. **Upload SOTK Document**: Upload Peraturan Mudir 002/2025 PDF for complete knowledge base
2. **WhatsApp Integration**: Connect with Twilio/WABA for production WhatsApp bot
3. **Production Deployment**: Deploy to Cloudflare Workers for edge computing
4. **Add More Agents**: Extend with specialized agents for specific tasks
5. **Dashboard Integration**: Connect with Corporate Planning dashboard

---

**Prepared by:** McKinsey Digital - Senior Expert Consultant

**Contact:** For implementation support, please reach out to the AI Practice team.